<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/architecture-patterns/1_residual_connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Residual block where filter size changes

In [21]:
input = keras.Input(shape=(32, 32, 3))

x = layers.Conv2D(filters = 32, kernel_size=3, activation="relu")(input)
residual = x

In [22]:
residual.shape

TensorShape([None, 30, 30, 32])

In [23]:
x = layers.Conv2D(filters = 64, kernel_size=3, activation="relu", padding='same')(x)

In [24]:
x.shape

TensorShape([None, 30, 30, 64])

In [26]:
#use Conv2D with kernel size 1 and required units to reshape to desired shape
residual = layers.Conv2D(64, kernel_size=1)(residual)

In [27]:
residual.shape

TensorShape([None, 30, 30, 64])

In [28]:
x = layers.add([x, residual])

## Residual block when using max pooling

In [36]:
inputs = keras.Input(shape=(32, 32, 3))

In [37]:
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)

In [38]:
residual = x 

In [39]:
residual.shape

TensorShape([None, 30, 30, 32])

In [40]:
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same')(x)

In [41]:
x.shape

TensorShape([None, 30, 30, 64])

In [42]:
x = layers.MaxPool2D(pool_size=(2,2), padding='same')(x)

In [43]:
x.shape

TensorShape([None, 15, 15, 64])

In [44]:
residual = layers.Conv2D(filters=64, kernel_size=1, strides=2)(residual)

In [45]:
residual.shape

TensorShape([None, 15, 15, 64])

In [47]:
x = layers.add([x, residual])

In [48]:
x.shape

TensorShape([None, 15, 15, 64])